# Dancer's business - 1. Data prepration

In [1]:
import pandas as pd
import numpy as np

# DataFrame to save

In [2]:
# Dataframe to merge everything
df = None

# Scale/adjustment factor data
Before we begin, let's import a few useful economic index data and census data.
Even though these data won't be used for my modeling, 
I will use them when we explore statistics to get a trend of business.

## Consumer Price Index (CPI)
CPI can be used to account for inflation. I downloaded numbers of interesting areas from the website below.
- Source: [U.S. Bureau of Labor Statistics (link is one of the example page)](https://www.bls.gov/regions/new-york-new-jersey/data/xg-tables/ro2xgcpiny1967.htm)
    - All area: Mean of half1 and half2 of "All items in U.S. city average, all urban consumers, not seasonally adjusted" (CUUR0000SA0)


In [3]:
cpi = pd.read_csv('data/CPI.csv')
display(cpi)

,year,NY,Chicago,Seattle,LA,SanFran,All
0,1990,138.500,131.700,126.800,135.900,132.100,130.6500
1,1991,144.800,137.000,134.100,141.400,137.900,136.2000
2,1992,150.000,141.100,139.000,146.500,142.500,140.3000
3,1993,154.500,145.400,142.900,150.300,146.300,144.5000
4,1994,158.200,148.600,147.800,152.300,148.700,148.2500
5,1995,162.200,153.300,152.300,154.600,151.600,152.3500
6,1996,166.900,157.400,157.500,157.500,155.100,156.8500
7,1997,170.800,161.700,163.000,160.000,160.400,160.5500
8,1998,173.600,165.000,167.700,162.300,165.500,163.0000
9,1999,177.000,168.400,172.800,166.100,172.500,166.6000


In [4]:
# Merge to df
df = pd.melt(cpi, id_vars=['year'], value_vars=cpi.columns[1:], 
            var_name='area', value_name='cpi') 

print(len(df)) # should be 32 years x 6 area = 192 rows
display(df.sample(5))

192


,year,area,cpi
108,2002,LA,182.2000
68,1994,Seattle,147.8000
189,2019,All,255.6575
31,2021,NY,292.3030
4,1994,NY,158.2000


## Cost of Living Index (CLI)
CPI may not account for actual living cost.
The CLI is closer to actual spend of living.
When we judge income level, we always consider the CLI to scale.

Unlike the CPI, finding CLI was difficult.
I found the CLI data of year 2010 from the Census.gov.
I will <font color=red>assume this number is staying same over years up to relative between cities.</font>

Also, it is <font color=red>not clear if this is calculated for metropolitan statistical area (broader) or only for city (smaller area). I'll assume the former, which is as same as the area division of wage statistics.</font>

In [5]:
cli = pd.read_csv('data/CLI.csv')
cli = cli[cli.area!='Percent']
display(cli)

,area,cli,Grocery,Housing,Utilities,Transportation,HealthCare,Etc
0,NY,216.7,154.3,386.7,169.6,120.3,130.2,145.7
1,Chicago,116.9,111.2,134.8,117.3,116.5,108.5,104.4
2,Seattle,121.4,115.1,140.3,85.7,118.8,119.9,119.1
3,LA,136.4,106.0,207.1,101.7,113.6,109.1,107.0
4,SanFran,164.0,111.9,281.0,94.5,113.0,117.0,124.3


Housing price is the dominant driving factor.

In [6]:
# Merge to df
df = df.merge(cli[['area','cli']], how = 'outer', on = ['area'])

print(len(df)) # should be 32 years x 6 area = 192 rows
display(df.sample(5))

192


,year,area,cpi,cli
27,2017,NY,268.520,216.7
132,1994,SanFran,148.700,164.0
184,2014,All,236.736,NaN
90,2016,Seattle,254.886,121.4
160,1990,All,130.650,NaN


## Census - U.S. and Metropolitan area population

- Source
    - National population
        - [Census.gov, 2010-2021](https://data.census.gov/cedsci/table?q=Age%20and%20Sex&tid=ACSDP1Y2010.DP05)
        - [Wikipedia, 1998-2009](https://en.wikipedia.org/wiki/Demographics_of_the_United_States)
    - Metropolitan statistical area population
        - [Census.gov, 2010-2021](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-metro-and-micro-statistical-areas.html)

### Metropolitan statistical area population data

In [7]:
# Check metropolitan area census dataset
census = pd.read_csv('data/census.csv')
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2797 entries, 0 to 2796
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CBSA               2797 non-null   int64  
 1   MDIV               141 non-null    float64
 2   STCOU              1840 non-null   float64
 3   NAME               2797 non-null   object 
 4   LSAD               2797 non-null   object 
 5   CENSUS2010POP      2797 non-null   int64  
 6   ESTIMATESBASE2010  2797 non-null   int64  
 7   POPESTIMATE2010    2797 non-null   int64  
 8   POPESTIMATE2011    2797 non-null   int64  
 9   POPESTIMATE2012    2797 non-null   int64  
 10  POPESTIMATE2013    2797 non-null   int64  
 11  POPESTIMATE2014    2797 non-null   int64  
 12  POPESTIMATE2015    2797 non-null   int64  
 13  POPESTIMATE2016    2797 non-null   int64  
 14  POPESTIMATE2017    2797 non-null   int64  
 15  POPESTIMATE2018    2797 non-null   int64  
 16  POPESTIMATE2019    2797 

In [8]:
# Select only interesting area

# Zipcode of Metropolitan Statistical Area
# LA has two zip codes because it has changed over years
zipcode_area = {31100:'LA',31080:'LA',41860:'SanFran',16980:'Chicago',35620:'NY',42660:'Seattle'}

lst=[]
for i in zipcode_area.keys():
    if i==31100:
        continue
    lst.append(census.loc[(census.CBSA==i)&(census.LSAD=='Metropolitan Statistical Area')])

census = pd.concat(lst)

display(census)

,CBSA,MDIV,STCOU,NAME,LSAD,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021
850,31080,NaN,NaN,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan Statistical Area,12828837,12828957,12838417,12925753,13013443,13097434,13166609,13234696,13270694,13278000,13249879,13214799,13173266.0,12997353.0
1307,41860,NaN,NaN,"San Francisco-Oakland-Berkeley, CA",Metropolitan Statistical Area,4335391,4335593,4343634,4395725,4455473,4519636,4584981,4647924,4688198,4712421,4726314,4731803,4739649.0,4623264.0
291,16980,NaN,NaN,"Chicago-Naperville-Elgin, IL-IN-WI",Metropolitan Statistical Area,9461105,9461537,9470634,9500870,9528090,9550194,9560430,9552554,9533662,9514113,9484158,9458539,9601605.0,9509934.0
1017,35620,NaN,NaN,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,18897109,18896277,18923407,19052774,19149689,19226449,19280929,19320968,19334778,19322607,19276644,19216182,20096413.0,19768458.0
1337,42660,NaN,NaN,"Seattle-Tacoma-Bellevue, WA",Metropolitan Statistical Area,3439809,3439808,3449241,3503891,3558829,3612347,3675160,3739654,3816355,3885579,3935179,3979845,4024730.0,4011553.0


In [9]:
# Select only interesting fields
census.drop(['MDIV','STCOU','LSAD','CENSUS2010POP','ESTIMATESBASE2010'],axis=1,inplace=True)

# Change name of dields
census.columns = ['CBSA','NAME',2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]

# Transpose
census = census.T

# Put area names as field names
census.columns = ['LA','SanFran','Chicago','NY','Seattle']

# Reset index
census.reset_index()

# Leave only yearly population rows
census.drop(['CBSA','NAME'],inplace=True)

display(census)

,LA,SanFran,Chicago,NY,Seattle
2010,12838417,4343634,9470634,18923407,3449241
2011,12925753,4395725,9500870,19052774,3503891
2012,13013443,4455473,9528090,19149689,3558829
2013,13097434,4519636,9550194,19226449,3612347
2014,13166609,4584981,9560430,19280929,3675160
2015,13234696,4647924,9552554,19320968,3739654
2016,13270694,4688198,9533662,19334778,3816355
2017,13278000,4712421,9514113,19322607,3885579
2018,13249879,4726314,9484158,19276644,3935179
2019,13214799,4731803,9458539,19216182,3979845


### National population data

In [10]:
# Check one example file of national population
demo = pd.read_csv('data/C2015.csv')

demo.info()
display(demo.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Label (Grouping)                        89 non-null     object
 1   United States!!Estimate                 84 non-null     object
 2   United States!!Margin of Error          84 non-null     object
 3   United States!!Percent                  84 non-null     object
 4   United States!!Percent Margin of Error  84 non-null     object
dtypes: object(5)
memory usage: 3.6+ KB


,Label (Grouping),United States!!Estimate,United States!!Margin of Error,United States!!Percent,United States!!Percent Margin of Error
0,SEX AND AGE,NaN,NaN,NaN,NaN
1,Total population,"321,418,821",*****,"321,418,821",(X)
2,Male,"158,167,834","±31,499",49.2%,±0.1
3,Female,"163,250,987","±31,500",50.8%,±0.1
4,Under 5 years,"19,793,807","±16,520",6.2%,±0.1


In [11]:
%%script false --no-raise-error
# If you already have data/usDemo.csv, this block can be skipped.

# Combine multiple year files of national population
df_save = []
for year in range(2010,2020):
    demo = pd.read_csv('data/C{0}.csv'.format(year))
    demo['year'] = year
    df_save.append(demo)
    
# Mave a csv file
df_save = pd.concat(df_save)

df_save.columns = ['label','estimate','estimate_err','pct','pct_err','year','estimate_err2']
df_save.to_csv('data/usDemo.csv', index=False)

In [12]:
census_national = pd.read_csv('data/usDemo.csv')
display(census_national)

,label,estimate,estimate_err,pct,pct_err,year,estimate_err2
0,SEX AND AGE,NaN,NaN,NaN,NaN,2010,NaN
1,Total population,"309,349,689",*****,"309,349,689",(X),2010,NaN
2,Male,"152,089,450","±27,325",49.2%,±0.1,2010,NaN
3,Female,"157,260,239","±27,325",50.8%,±0.1,2010,NaN
4,Under 5 years,"20,133,943","±20,568",6.5%,±0.1,2010,NaN
...,...,...,...,...,...,...,...
880,Total housing units,"139,686,209",NaN,(X),(X),2019,"±6,973"
881,"CITIZEN, VOTING AGE POPULATION",NaN,NaN,NaN,NaN,2019,NaN
882,"Citizen, 18 and over population","235,418,734",NaN,"235,418,734",(X),2019,"±159,764"
883,Male,"114,206,194",NaN,48.5%,±0.1,2019,"±98,225"


In [13]:
# Leave data to use only
# Change format to merge with metropolitan census dataframe

population=[]
years=[]

# make year:population dictionary
for year in range(2010,2020):
    pop = int(''.join(census_national[(census_national.label.str.contains('Total population'))\
                                      &(census_national.year==year)].iloc[0].estimate.split(',')))
    population.append(pop)
    years.append(year)
    

census_national = pd.DataFrame({'year':years,'All':population})

In [14]:
# Add extra years

census_national_other = pd.DataFrame([
    [2020, 331501080],
    [2021, 331893745],
    [1998, 275854000],
    [1999, 279040000],
    [2000, 282172000],
    [2001, 285082000],
    [2002, 287804000],
    [2003, 290326000],
    [2004, 293046000],
    [2005, 295753000],
    [2006, 298593000],
    [2007, 301580000],
    [2008, 304375000],
    [2009, 307007000]], 
    columns=['year','All'])


census_national = census_national.append(census_national_other, ignore_index=True)


census_national = census_national.sort_values(by=['year'])

display(census_national)

,year,All
12,1998,275854000
13,1999,279040000
14,2000,282172000
15,2001,285082000
16,2002,287804000
17,2003,290326000
18,2004,293046000
19,2005,295753000
20,2006,298593000
21,2007,301580000


### Merge national population to metropolitan population

In [15]:
census = pd.merge(census_national, census, how='outer', right_index=True, left_on='year')

# Change data type to numeric
census = census.apply(pd.to_numeric)

census.info()
display(census)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 12 to 11
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     24 non-null     int64  
 1   All      24 non-null     int64  
 2   LA       12 non-null     float64
 3   SanFran  12 non-null     float64
 4   Chicago  12 non-null     float64
 5   NY       12 non-null     float64
 6   Seattle  12 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 1.5 KB


,year,All,LA,SanFran,Chicago,NY,Seattle
12,1998,275854000,NaN,NaN,NaN,NaN,NaN
13,1999,279040000,NaN,NaN,NaN,NaN,NaN
14,2000,282172000,NaN,NaN,NaN,NaN,NaN
15,2001,285082000,NaN,NaN,NaN,NaN,NaN
16,2002,287804000,NaN,NaN,NaN,NaN,NaN
17,2003,290326000,NaN,NaN,NaN,NaN,NaN
18,2004,293046000,NaN,NaN,NaN,NaN,NaN
19,2005,295753000,NaN,NaN,NaN,NaN,NaN
20,2006,298593000,NaN,NaN,NaN,NaN,NaN
21,2007,301580000,NaN,NaN,NaN,NaN,NaN


In [16]:
# Merge to df
census = pd.melt(census, id_vars=['year'], value_vars=cpi.columns[1:], 
        var_name='area', value_name='population')

df = df.merge(census, how='outer', on= ['area','year'])

print(len(df)) # should be 32 years x 6 area = 192 rows
display(df.sample(5))

192


,year,area,cpi,cli,population
160,1990,All,130.65,NaN,NaN
30,2020,NY,282.92,216.7,20096413.0
166,1996,All,156.85,NaN,NaN
136,1998,SanFran,165.50,164.0,NaN
33,1991,Chicago,137.00,116.9,NaN


# U.S. market data

I crated "data/rev.csv" file by combining U.S. markets statistics from multiple sources.
Data sources are
- U.S. dance studio 
    - market size: [Statista](https://www.statista.com/statistics/1175824/dance-studio-industry-market-size-us/)
    - number of businesses: [IBISWorld](https://www.ibisworld.com/industry-statistics/number-of-businesses/dance-studios-united-states/)
    - number of employees: [IBISWorld](https://www.ibisworld.com/industry-statistics/employment/dance-studios-united-states/)
    - wages: [IBISWorld](https://www.ibisworld.com/industry-statistics/wages/dance-studios-united-states/)
- U.S. fitness and recreational sports centers 
    - revenue: [FRED, cited U.S. Bureau of Labor Statistics](https://fred.stlouisfed.org/series/REVEF71394ALLEST).

In [17]:
# Read data file
rev = pd.read_csv('data/rev.csv')

# Set every money scale to billion dollars
rev.fitness = rev.fitness/1000
rev.studio_wage = rev.studio_wage/1000

display(rev)
# fitness: U.S. fitness and recreational sports center revenue
# dance_studio: U.S. dance studio revenue
# studio_num: number of buinesses of U.S. dance studio
# studio_emp: number of employees of U.S. dance studio
# studio_wage: total wage of U.S. dance studio

,year,fitness,dance_studio,studio_num,studio_emp,studio_wage
0,1998,10.797,NaN,NaN,NaN,NaN
1,1999,11.777,NaN,NaN,NaN,NaN
2,2000,12.543,NaN,NaN,NaN,NaN
3,2001,13.542,NaN,NaN,NaN,NaN
4,2002,14.987,NaN,NaN,NaN,NaN
5,2003,16.287,NaN,NaN,NaN,NaN
6,2004,17.174,NaN,NaN,NaN,NaN
7,2005,18.286,NaN,NaN,NaN,NaN
8,2006,19.447,NaN,NaN,NaN,NaN
9,2007,21.416,NaN,NaN,NaN,NaN


In [18]:
# Merge to df
df= df.merge(rev, how = 'outer', on= ['year'])

# for convenience
df.loc[df.area.isna(),'area']='All'

print(len(df)) # should be 32 years x 6 area + 1 year row = 193 rows
display(df.sample(3))

193


,year,area,cpi,cli,population,fitness,dance_studio,studio_num,studio_emp,studio_wage
4,1990,SanFran,132.100,164.0,NaN,NaN,NaN,NaN,NaN,NaN
145,2014,Chicago,228.468,116.9,9560430.0,27.001,3.42,52942.0,99696.0,0.9504
190,2021,SanFran,309.721,164.0,4623264.0,NaN,3.72,66266.0,120081.0,1.1531


# Employee statistics data
- Source: [Occupational Employment and Wage Statistics provided by U.S. Bureau of Labor Statistics](https://www.bls.gov/oes/tables.htm).

This website provides a table of employment statistics (wage, number of employee, etc) of different area of each year.
I downloaded each year's file, and they will be cleaned and concatenated.

## Explanation of fields

Here are definitions of each field. Not explained field is not used in this analysis.

### Area identifier
- area: area code 
- area_name(title):	Area name 

### Job identifier
- occ_code: The 6-digit Standard Occupational Classification (SOC) code or OEWS-specific code for the occupation 
- occ_title: SOC title or OEWS-specific title for the occupation

### Number of employee
- tot_emp: Estimated total employment rounded to the nearest 10 (excludes self-employed).
- emp_prse:	Percent relative standard error (PRSE) for the employment estimate. PRSE is a measure of sampling error, expressed as a percentage of the corresponding estimate. Sampling error occurs when values for a population are estimated from a sample survey of the population, rather than calculated from data for all members of the population. Estimates with lower PRSEs are typically more precise in the presence of sampling error.

### Wage
- h_mean: Mean hourly wage
- a_mean: Mean annual wage 

- mean_prse: Percent relative standard error (PRSE) for the mean wage estimate.
- h_pct10: Hourly 10th percentile wage
- h_pct25: Hourly 25th percentile wage
- h_median: Hourly median wage (or the 50th percentile)
- h_pct75: Hourly 75th percentile wage
- h_pct90: Hourly 90th percentile wage

- a_pct10: Annual 10th percentile wage
- a_pct25: Annual 25th percentile wage
- a_median: Annual median wage (or the 50th percentile)
- a_pct75: Annual 75th percentile wage
- a_pct90: Annual 90th percentile wage

In [19]:
# Let's check how each file looks like
sample = pd.read_excel('data/2010.xls')

sample.info()
display(sample.sample(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7940 entries, 0 to 7939
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PRIM_STATE    7940 non-null   object
 1   AREA          7940 non-null   int64 
 2   AREA_NAME     7940 non-null   object
 3   OCC_CODE      7940 non-null   object
 4   OCC_TITLE     7940 non-null   object
 5   GROUP         253 non-null    object
 6   TOT_EMP       7940 non-null   object
 7   EMP_PRSE      7940 non-null   object
 8   JOBS_1000     7940 non-null   object
 9   LOC QUOTIENT  7940 non-null   object
 10  H_MEAN        7940 non-null   object
 11  A_MEAN        7940 non-null   object
 12  MEAN_PRSE     7940 non-null   object
 13  H_PCT10       7940 non-null   object
 14  H_PCT25       7940 non-null   object
 15  H_MEDIAN      7940 non-null   object
 16  H_PCT75       7940 non-null   object
 17  H_PCT90       7940 non-null   object
 18  A_PCT10       7940 non-null   object
 19  A_PCT2

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
3326,IL,16980,"Chicago-Naperville-Joliet, IL-IN-WI",43-3061,Procurement Clerks,NaN,1640,8,0.392,0.677,...,18.82,21.89,25.56,27480,32340,39150,45530,53160,NaN,NaN
6638,TX,19100,"Dallas-Fort Worth-Arlington, TX",19-3091,Anthropologists and Archeologists,NaN,90,33.8,0.031,0.772,...,23.59,30.58,45.45,34460,39710,49070,63600,94540,NaN,NaN


In [20]:
# jobs in interest
    # dancer: dancer
    # choreo: choreographer, 
    # fit_trainer: fitness trainer/instructer
    # rec_worker: recreational worker, 
    # all_jobs: all jobs sum/mean
    
code_job = {'27-2031':'dancer','27-2032':'choreo',
            '39-9031':'fit_trainer','39-9032':'rec_worker',
            '00-0000':'all_jobs'}

In [21]:
%%script false --no-raise-error
# This block combines multiple wage data files, then generate a single csv file.
# If you already have data/wage.csv, this block can be skipped. It takes time to run.


df_save = []
for year in range(2003,2022):
    
    print(year)
    metro = None # metropolitan area statistics data
    national = None # national statistics data
    
    try:
        if year>2004:
            metro = pd.read_excel('data/'+str(year)+'.xls')
        national = pd.read_excel('data/'+str(year)+'nat.xls')
    except:
        if year>2004:
            metro = pd.read_excel('data/'+str(year)+'.xlsx')
        national = pd.read_excel('data/'+str(year)+'nat.xlsx')

    if year>2004:
        metro.columns = metro.columns.str.strip().str.lower()
    national.columns = national.columns.str.strip().str.lower()
    
    # unify feature names in all years
    if year>2004:
        metro.rename(columns={'area_title':'area_name'},inplace=True)
        
    # LA area code changed
    area_la = 31100
    if year>2014:
        area_la=31080

    if year>2004:
        # Select metropolitan area in interest
        metro = metro.loc[(metro.area==area_la) | (metro.area==41860) | (metro.area==16980) | 
                        (metro.area==35620) | (metro.area==42660)]

        # Select occupation in interest
        metro = metro.loc[(metro.occ_code=='27-2031') | (metro.occ_code=='27-2032') | 
                          (metro.occ_code=='39-9031') | (metro.occ_code=='39-9032') | 
                          (metro.occ_code=='00-0000')]
        # Change zip code to the unique area names
        metro['area']=metro.apply(lambda x: zipcode_area[x['area']], axis=1)
        
    # Select occupation in interest
    national = national.loc[(national.occ_code=='27-2031') | (national.occ_code=='27-2032') |
                            (national.occ_code=='39-9031') | (national.occ_code=='39-9032') |                          
                            (national.occ_code=='00-0000')]

    


    # To match columns with metropolitan dataframe
    national['area'] = 'All'
    national['area_name'] = 'U.S. all'
    
    # Keep only columns to use
    if year>2004:
        metro = metro[['area', 'area_name', 'occ_code', 'occ_title', 
           'tot_emp', 'emp_prse', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10',
           'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25',
           'a_median', 'a_pct75', 'a_pct90']]

    national = national[['area', 'area_name', 'occ_code', 'occ_title', 
   'tot_emp', 'emp_prse', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10',
   'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25',
   'a_median', 'a_pct75', 'a_pct90']]
    
    
    # comebine national data to metropolitan data
    emp=None
    
    if year>2004:
        emp = pd.concat([national,metro], ignore_index=True)
    else:
        emp = national

    # add year
    emp['year']=year

    # add the unique occupation name
    emp['occ']=emp.apply(lambda x: code_job[x['occ_code']], axis=1)

    # Cleaning
    emp.replace('**',np.nan,inplace=True)
    emp.replace('*',np.nan,inplace=True)

    # Append to a list to save
    df_save.append(emp)

# Mave a csv file
df_save = pd.concat(df_save)
df_save.to_csv('data/emp.csv', index=False)

In [22]:
# Check data is prepared as intended
emp = pd.read_csv('data/emp.csv')

emp.info()

# Confirm if city and occupation labels are correct
print("Check area names are correctly marked ------ ")
for x in zipcode_area.values():
    print(x,emp[emp.area==x].area_name.unique())
    
print("\n Check occupation names are correctly marked ------ ")    
for x in code_job.values():
    print(x,emp[emp.occ==x].occ_title.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area       497 non-null    object 
 1   area_name  497 non-null    object 
 2   occ_code   497 non-null    object 
 3   occ_title  497 non-null    object 
 4   tot_emp    462 non-null    float64
 5   emp_prse   462 non-null    float64
 6   h_mean     491 non-null    float64
 7   a_mean     402 non-null    float64
 8   mean_prse  491 non-null    float64
 9   h_pct10    491 non-null    float64
 10  h_pct25    491 non-null    float64
 11  h_median   491 non-null    float64
 12  h_pct75    491 non-null    float64
 13  h_pct90    491 non-null    float64
 14  a_pct10    402 non-null    float64
 15  a_pct25    402 non-null    float64
 16  a_median   402 non-null    float64
 17  a_pct75    402 non-null    float64
 18  a_pct90    402 non-null    float64
 19  year       497 non-null    int64  
 20  occ       

In [23]:
# Since area and occupations are correctly marked, let's remove them
emp.drop(['area_name','occ_code','occ_title'],axis=1,inplace=True)
print(len(emp)) # 17 years x 6 area x 5 jobs + 2 years x 1 area x 5 jobs = 520 rows, if no missing record
display(emp.sample(5))

497


,area,tot_emp,emp_prse,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,year,occ
212,Seattle,5110.0,16.8,20.64,42920.0,4.1,10.26,15.12,20.35,24.31,32.23,21340.0,31450.0,42320.0,50570.0,67030.0,2011,fit_trainer
286,Chicago,4411860.0,0.3,24.23,50410.0,0.8,9.08,11.19,18.07,30.75,46.01,18890.0,23280.0,37590.0,63950.0,95700.0,2014,all_jobs
425,Chicago,14360.0,6.9,24.11,50150.0,3.0,9.90,13.01,22.83,30.58,43.08,20590.0,27060.0,47480.0,63610.0,89610.0,2019,fit_trainer
89,NY,1420.0,23.7,18.46,NaN,9.0,7.46,9.30,15.26,25.21,31.55,NaN,NaN,NaN,NaN,NaN,2007,dancer
454,LA,110.0,35.3,20.34,NaN,7.7,13.20,14.12,16.93,22.49,32.35,NaN,NaN,NaN,NaN,NaN,2020,dancer


In [24]:
# Merge to df
df= df.merge(emp, how = 'outer', on= ['area','year'])

print(len(df)) # 520 rows, if no missing record +
                # 89 rows, 15 years x 5 area + 14 years x 1 area = 609
display(df.sample(5))

586


,year,area,cpi,cli,population,fitness,dance_studio,studio_num,studio_emp,studio_wage,...,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,occ
464,2017,LA,256.210,136.4,13278000.0,33.042,3.87,58515.0,114075.0,1.1448,...,14.59,21.34,29.33,38.37,23570.0,30340.0,44390.0,61010.0,79800.0,fit_trainer
314,2012,Seattle,238.663,121.4,3558829.0,24.051,3.22,47269.0,90668.0,0.9026,...,10.24,12.04,15.43,20.71,19560.0,21310.0,25030.0,32100.0,43070.0,rec_worker
79,2003,Chicago,184.500,116.9,NaN,16.287,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,2009,NY,236.825,216.7,NaN,21.842,NaN,NaN,NaN,NaN,...,12.71,23.50,34.54,44.10,17000.0,26430.0,48890.0,71840.0,91730.0,fit_trainer
572,2021,LA,289.244,136.4,12997353.0,NaN,3.72,66266.0,120081.0,1.1531,...,19.78,36.86,36.86,36.88,41140.0,41140.0,76660.0,76660.0,76700.0,choreo


In [28]:
# It was expected to have  # 17 years x 6 area x 5 jobs + 2 years x 1 area x 5 jobs = 520 job records
# But we have 497 rows.
# Which records are missing?

for i in range(2006,2022):
    for j in df.area.unique():
        if df.loc[(df.area==j)&(df.year==i)].occ.nunique()!=5:
            print(i,j, set(df.occ.unique()) - set(df.loc[(df.area==j)&(df.year==i)].occ.unique())) 

2006 NY {'dancer'}
2009 Seattle {'choreo'}
2010 Seattle {'dancer'}
2011 Seattle {'choreo', 'dancer'}
2012 Seattle {'choreo', 'dancer'}
2013 Seattle {'dancer'}
2014 Seattle {'dancer'}
2015 Seattle {'dancer'}
2016 Seattle {'dancer'}
2018 Seattle {'choreo'}
2019 NY {'choreo'}
2019 Seattle {'choreo'}
2019 SanFran {'choreo'}
2020 NY {'choreo'}
2020 Chicago {'choreo'}
2020 Seattle {'choreo'}
2020 SanFran {'choreo'}
2021 Chicago {'choreo', 'dancer'}
2021 Seattle {'dancer'}


Some missing records of dancer's income data in multiple cities and years. 
Of course, dancers are hard to track!
Let's not bother to handle the missing data here for now.
If dancer's statistics are irregular over year, we can't impute its empty record anyway.

# Save organized dataset
Now, we have all data prepared. 
Let's save it for next steps.

In [26]:
# for convenience
df.loc[df.occ.isna(),'occ']='all_jobs'

In [27]:
%%script false --no-raise-error
# If you already have data/dance.csv, this block can be skipped.
df.to_csv('data/dance1.csv',index=False)